## HW 2 - Algorithms and Applications in Social Networks

Names & IDs:

Yonatan Voikhansky, 315398339

Ariel Ireni, 313914970

In [14]:
from scipy.stats.distributions import nakagami
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np

## Question 1
### Part A

In [15]:
def get_edge_betweenness(G):
    betweennes_dict = dict()
    for edge in G.edges:
        count_e_in_path = 0
        for node_s in G.nodes:
            for node_t in G.nodes:
                if node_t != node_s:
                    all_shortest_paths = list(nx.all_shortest_paths(G, node_s, node_t))
                    sum_in_paths = 0
                    for path in all_shortest_paths:
                        for i in range(0, len(path)-1):
                            for j in range(i+1, len(path)):
                                if edge == (path[i], path[j]):
                                    sum_in_paths += 1
                    
                    count_e_in_path += sum_in_paths / len(all_shortest_paths)

        betweennes_dict[edge] = count_e_in_path
    return betweennes_dict

def test_Q2a():
    G = nx.gnp_random_graph(n=10, p=.3)
    result = get_edge_betweenness(G)
    # print(nx.edge_betweenness_centrality(G, normalized=False) == result)

test_Q2a()

# TODO: handle numerial issues

In [16]:
def Newman_Girvan(G, k):
    G = G.copy()
    while len(G.edges) > 0:
        # cent_dict = get_edge_betweenness(G)
        cent_dict = nx.edge_betweenness(G)
        max_edge = max(cent_dict, key=cent_dict.get)
        G.remove_edge(*max_edge)
        if nx.number_connected_components(G) == k:
            return list(nx.connected_components(G))

G = nx.karate_club_graph()
Newman_Girvan(G, 2)


[{0, 1, 3, 4, 5, 6, 7, 10, 11, 12, 13, 16, 17, 19, 21},
 {2, 8, 9, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33}]

### Part B

In [84]:
import requests

def get_biggest_connected_component():
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
    }

    response = requests.get("http://slavanov.com/teaching/sn1718b/data/communities.txt", headers=headers)
    data = response.text
    data = data.split("\n")
    edges = []
    for line in data:
        line = line.split(" ")
        edges.append(line)
    edges = edges[:-1]

    G = nx.from_edgelist(edges)
    largest_cc = max(nx.connected_components(G), key=len)
    G_comp = nx.subgraph(G, list(largest_cc))
    return G_comp

def Newman_Girvan_largest_component():
    G_comp = get_biggest_connected_component()
    communities = Newman_Girvan(G_comp, 3)
    return [list(comm) for comm in communities]


lst = Newman_Girvan_largest_component()
for comm in lst:
    print("Community: ")
    print(comm)

Community: 
['260', '342', '84', '107', '322', '21', '280', '127', '122', '330', '315', '180', '200', '106', '148', '24', '203', '132', '57', '208', '55', '329', '336', '295', '288', '64', '229', '281', '120', '166', '7', '303', '188', '56', '58', '300', '161', '163', '196', '207', '277', '246', '172', '117', '141', '101', '45', '271', '258', '332', '36', '257', '65', '235', '53', '129', '344', '30', '339', '224', '27', '103', '249', '187', '73', '31', '85', '269', '198', '250', '272', '211', '346', '9', '270', '34', '171', '113', '340', '16', '222', '13', '105', '221', '231', '156', '237', '164', '299', '285', '341', '72', '290', '268', '100', '158', '345', '136', '284', '318', '60', '274', '169', '98', '168', '324', '67', '254', '228', '75', '294', '184', '29', '325', '323', '186', '194', '176', '48', '189', '347', '79', '77', '314', '265', '76', '146', '191', '276', '320', '212', '286', '204', '130', '54', '199', '133', '197', '128', '304', '94', '81', '165', '50', '202', '173', '87

## Question 2
### Part A

In [78]:
def get_k_clique_communities(G, k):
	# find all maximal cliques
	maximal_cliques = list(nx.find_cliques(G))
	
	# create clique overlap matrix
	n = len(maximal_cliques)
	overlap_matrix = [[0 for col in range(n)] for row in range(n)]
	for i in range(len(maximal_cliques)):
		c_1 = maximal_cliques[i]
		
		for j in range(len(maximal_cliques)):
			count_overlap = 0
			c_2 = maximal_cliques[j]
			for node in c_2:
				if node in c_1: 
					count_overlap += 1

			overlap_matrix[i][j] = count_overlap

	# treshold matrix with k-1
	for i in range(len(overlap_matrix)):
		for j in range(len(overlap_matrix)):
			if i == j:
				if overlap_matrix[i][j] < k:
					overlap_matrix[i][j] = 0
				else:
					overlap_matrix[i][j] = 1
			else:
				if overlap_matrix[i][j] < k-1:
					overlap_matrix[i][j] = 0
				else:
					overlap_matrix[i][j] = 1	
					
    	# communities are connected components
	A = np.array(overlap_matrix)

	G_c = nx.from_numpy_matrix(A, create_using=nx.MultiGraph)
	
	to_be_removed = [x for  x in G_c.nodes() if G_c.degree(x) < 1]
	for x in to_be_removed:
		G_c.remove_node(x)
	
	cliques_comp = list(nx.connected_components(G_c))

	# [[0, 1, 4] [5]]
	communities = []

	for comp in list(cliques_comp):
		curr_community = []
		for clique in comp:
			curr_community = curr_community + maximal_cliques[clique]
	
		communities.append(set(curr_community))

	return communities


In [81]:
def test_2a():
	G = nx.from_edgelist([(1,2), (1,3), (1,4), (2,4), (3,4), (2,3), (2,5), (2,6), (5,6), (4,6), (6,7), (4,7), (6,8), (7,8), (4,8), (3,9), (9,10), (10,4), (10,8), (10,6), (9,6), (3,10), (4,9), (8,9)])
	print(get_k_clique_communities(G, 4))

test_2a()

[{1, 2, 3, 4}, {3, 4, 6, 7, 8, 9, 10}]


#### Part B

In [90]:
from networkx.algorithms.community import k_clique_communities

def k_clique_largest_component():
	G_comp = get_biggest_connected_component()
	return get_k_clique_communities(G_comp, 4)

def test_2b():
	our_communities = k_clique_largest_component()
	G_comp = get_biggest_connected_component()
	their_communities = list(k_clique_communities(G_comp, 4))
	assert len(our_communities) == len(their_communities)
	for i in range(len(our_communities)):
		assert sorted(our_communities[i]) == sorted(their_communities[i])


test_2b()